# Glacier runoff in the Alps

If necessary, install the DTCG API with:

```
!pip install 'dtcg[jupyter] @ git+https://github.com/DTC-Glaciers/dtcg'
```

In a cell below.

In [ ]:
import dtcg
import dtcg.interface.gateway
import dtcg.interface.plotting

A user interacts with DTCG only through API requests.
This means a single centralised framework can handle requests from Jupyter notebooks, websites, and cli wrappers.
Flows are simplified for non-technical users, and can be customised for more advanced use cases.
It also prevents significant changes to the DTCG API interfering with existing flows, as little to no backend code is exposed to the user.

In [ ]:
# User selects these via dropdown menus
subregion_name = "vent_rofenache"
glacier_name = "Hintereisferner"

API queries are extensible, as long as they conform to the OpenAPI standard.
For selecting a subregion, this is what an API query could look like:

In [ ]:
# A query might look like this
user_query_params = {
    "action": "select_subregion",
    "region_name": "Central Europe",
    "subregion_name": subregion_name,
    "glacier_name": glacier_name,
    "shapefile_path": "nested_catchments_oetztal/nested_catchments_oetztal.shx",
    "oggm_params": {
        "use_multiprocessing": True,
        "rgi_version": "62",
        "store_model_geometry": True,
    },
}

A user can also select specific glaciers:

In [ ]:
# A query might look like this
user_query_params = {
    "action": "select_glacier",
    "region_name": "Central Europe",
    "subregion_name": subregion_name,
    "glacier_name": glacier_name,
    "shapefile_path": "nested_catchments_oetztal/nested_catchments_oetztal.shx",
    "oggm_params": {
        "use_multiprocessing": True,
        "rgi_version": "62",
        "store_model_geometry": True,
    },
}
handler = dtcg.interface.gateway.GatewayHandler(query=user_query_params)
assert handler.response["response_code"] == "200"
data = handler.response["data"]

The API is very flexible: it can pass OGGM parameters directly to OGGM, and to preserve bandwidth a response can be customised to contain as little data as needed.

Components are standalone, and can be combined into different dashboards.

For this example, the response data are synthesised into interactive plots.
In a dashboard setting, these plots can also support real-time data.

In [ ]:
dashboard = dtcg.interface.plotting.HoloviewsDashboard()
dashboard.plot_graph.plot_mass_balance(
    observations=data["runoff_data"]["wgms"],
    mass_balance=data["runoff_data"]["mass_balance"],
    name=handler.query.glacier_name,
)

All figures are generated using Holoviews, with Bokeh as a backend. Components can be accessed individually, or implemented into a dashboard. These can then be implemented into a frontend interface using FastAPI and Panel for web, and FastAPI and Celery for Jupyter notebooks.

In [ ]:
dashboard = dtcg.interface.plotting.HoloviewsDashboardL2()
dashboard.plot_runoff_dashboard(
    data=data,
    subregion_name=handler.query.subregion_name,
    glacier_name="",
)